In [ ]:
import json

# Open the file and load the JSON data
with open('json/map.json', 'r', encoding='utf-8') as file:
    details = json.load(file)

# Now 'data' contains the content of your JSON file
print(details["12"])

Napewno odrzucamy wszytskie cechy, które zawierają ścieżki do jakichś plików typu logo, itp. a więc odpadają:
- logo_path
- poster_path
- backdrop_path<br>
Dodatkowo odrzucamy:
- id
- imdb_id
- id w production_companies<br>

Uważam też, że nie potrzebne są do klasyfikacji rozdzielenie informacji o krajach na "name" i "iso...", dlatego z racji krótszej reprezentacji wszędzie, gdzie pojawi się kraj będe korzystał ze skróconego ticker'a iso, np. <br>
"production_countries": [
        {
            "iso_3166_1": "DE",
            "name": "Germany"
        },
        {
            "iso_3166_1": "US",
            "name": "United States of America"
        }
    ],<br>
Przekształcam na zwykłą listę<br>
"production_countries": [DE, US]

"production_company" będzie uproszczone i będzie reprezentowane tylko przez "id" i "origin_country". "id" będzie podbijało miarę similarity, gdy będzie to samo "production_company", równie dobrze mogłobyć tu wzięte "name", ale z natury liczby łatwiej i szybciej się porównuje. Z belongs_to_collection weźmiemy tylko id, wartość porównania będzie 0-1, albo należą do tej samej kolekcji albo nie i nie ma to wpływu na klasyfikacje, nie ma sensu porównywać stringa.

Bazując na powyższym należy stworzyć takie wektory cech dla każdego filmu w plikach. Dla czytelności stworzona zostanie klasa Movie

In [2]:
movie_details = json.loads(formatted_movie_details)

In [3]:
movie_details["adult"]

False

In [4]:
class Movie:
    def __init__(self, adult, belongs_to_collection, budget, cast, director, genres, original_language, original_title, overview, popularity, production_companies, production_countries, 
                 release_date, revenue, runtime, spoken_languages, status, tagline, title, video, vote_average, vote_count):
        self.adult = adult
        self.belongs_to_collection = belongs_to_collection
        self.budget = budget
        self.cast = cast
        self.director = director
        self.genres = genres
        self.original_language = original_language
        self.original_title = original_title
        self.overview = overview
        self.popularity = popularity
        self.production_companies = production_companies
        self.production_countries = production_countries
        self.release_date = release_date
        self.revenue = revenue
        self.runtime = runtime
        self.spoken_languages = spoken_languages
        self.status = status
        self.tagline = tagline
        self.title = title
        self.video = video
        self.vote_average = vote_average
        self.vote_count = vote_count
    def __str__(self):
        return ("adult = " + str(self.adult) + "\n" + "belongs_to_collection = " + str(self.belongs_to_collection) + "\n" + "budget = " + str(self.budget) + "\n" +
        "cast = " + str(self.cast) + "\n" + "director = " + str(self.director) + "\n" +
        "genres = " + str(self.genres) + "\n" + "original_language = " + str(self.original_language) + "\n" + "original_title = " + str(self.original_title) + "\n" +
        "overview = " + str(self.overview) + "\n" + "popularity = " + str(self.popularity) + "\n" + "production_companies = " + str(self.production_companies) + "\n" +
        "production_countries = " + str(self.production_countries) + "\n" + "release_date = " + str(self.release_date) + "\n" + "revenue = " + str(self.revenue) + "\n" +
        "runtime = " + str(self.runtime) + "\n" + "spoken_languages = " + str(self.spoken_languages) + "\n" + "status = " + str(self.status) + "\n" +
        "tagline = " + str(self.tagline) + "\n" + "title = " + str(self.title) + "\n" + "video = " + str(self.video) + "\n" + "vote_average = " + str(self.vote_average) + "\n" +
        "vote_count = " + str(self.vote_count))

In [17]:
def getCast(movie):
    list_of_actors = []
    for actor in movie["credits"]["cast"]:
        list_of_actors.append(actor["name"]) 
    return list_of_actors

In [18]:
getCast(details["12"])

['Albert Brooks',
 'Ellen DeGeneres',
 'Alexander Gould',
 'Willem Dafoe',
 'Geoffrey Rush',
 'Brad Garrett',
 'Allison Janney',
 'Austin Pendleton',
 'Stephen Root',
 'Vicki Lewis']

In [5]:
# ta funkcja przyjmuje movie jako json, zwraca listę tickerów państw, np. EN, US
def productionCountriesConverter(movie):
    result = []
    for country in movie_details["production_countries"]:
        country_name = country["iso_3166_1"]
        result.append(country_name)
    return result

In [6]:
productionCountriesConverter(movie_details)

['GB']

In [7]:
# ta funkcja przyjmuje movie jako json, zwraca id kolekcji
def bellongsToCollectionConverter(movie):
    return movie_details["belongs_to_collection"]["id"]

In [8]:
bellongsToCollectionConverter(movie_details)

645

In [9]:
# ta funkcja przyjmuje movie jako json, zwraca listę tickerów państw, np. EN, US
def spokenLanguagesConverter(movie):
    result = []
    for country in movie_details["spoken_languages"]:
        country_name = country["iso_639_1"]
        result.append(country_name)
    return result

In [10]:
spokenLanguagesConverter(movie_details)

['de', 'en', 'ru', 'es']

In [11]:
# ta funkcja przyjmuje movie jako json, zwraca listę list [name, origin_country]
def productionCompaniesConverter(movie):
    result = []
    for company in movie_details["production_companies"]:
        comp = []
        comp.append(company["name"])
        comp.append(company["origin_country"])
        result.append(comp)
    return result

In [12]:
productionCompaniesConverter(movie_details)

[['EON Productions', 'GB']]

In [17]:
#przyjmuje movie jako json, zwraca obiekt z już przeformatowanymi wartościami
def movieConverter(movie):
    if movie["adult"] == False:
        adult = 0
    else:
        adult = 1
    if movie["video"] == False:
        video = 0
    else:
        video = 1
    return Movie(
        adult,
        bellongsToCollectionConverter(movie),
        movie["budget"],
        genresConverter(movie),
        movie["original_language"],
        movie["original_title"],
        movie["overview"],
        movie["popularity"],
        productionCompaniesConverter(movie),
        productionCountriesConverter(movie),
        movie["release_date"],
        movie["revenue"],
        movie["runtime"],
        spokenLanguagesConverter(movie),
        movie["status"],
        movie["tagline"],
        movie["title"],
        video,
        movie["vote_average"],
        movie["vote_count"]
    )

In [29]:
movieObject = movieConverter(movie_details)
print(movieObject)

adult = 0
belongs_to_collection = 645
budget = 27500000
genres = ['Adventure', 'Action', 'Thriller']
original_language = en
original_title = Octopussy
overview = James Bond is sent to investigate after a fellow “00” agent is found dead with a priceless Indian Fabergé egg. Bond follows the mystery and uncovers a smuggling scandal and a Russian General who wants to provoke a new World War.
popularity = 37.282
production_companies = [['EON Productions', 'GB']]
production_countries = ['GB']
release_date = 1983-06-05
revenue = 187500000
runtime = 131
spoken_languages = ['de', 'en', 'ru', 'es']
status = Released
tagline = James Bond's all time high.
title = Octopussy
video = 0
vote_average = 6.321
vote_count = 1781


In [19]:
import json

# Open the file and load the JSON data
with open('genres.json', 'r', encoding='utf-8') as file:
    genres = json.load(file)

# Now 'data' contains the content of your JSON file
print(type(genres))

<class 'dict'>


In [20]:
for elem in genres["genres"]:
    print(elem)

{'index': 0, 'name': 'Action'}
{'index': 1, 'name': 'Adventure'}
{'index': 2, 'name': 'Animation'}
{'index': 3, 'name': 'Comedy'}
{'index': 4, 'name': 'Crime'}
{'index': 5, 'name': 'Documentary'}
{'index': 6, 'name': 'Drama'}
{'index': 7, 'name': 'Family'}
{'index': 8, 'name': 'Fantasy'}
{'index': 9, 'name': 'History'}
{'index': 10, 'name': 'Horror'}
{'index': 11, 'name': 'Music'}
{'index': 12, 'name': 'Mystery'}
{'index': 13, 'name': 'Romance'}
{'index': 14, 'name': 'Science Fiction'}
{'index': 15, 'name': 'TV Movie'}
{'index': 16, 'name': 'Thriller'}
{'index': 17, 'name': 'War'}
{'index': 18, 'name': 'Western'}


In [21]:
GENRES = genres["genres"]

In [25]:
# ta funkcja przyjmuje movie jako json, zwraca listę gatunków z 1 jak występował dany gatunek, 0 jak nie
def genresFeatureVectorCreator(movie):
    indexes = [] # lista do przechowywania indexów, który gatunek wystąpił
    for genre in movie["genres"]:
        for genreFromJson in GENRES:
            if genre["name"] == genreFromJson["name"]:
                indexes.append(genreFromJson["index"])
    genre_feature_vector = []
    for index in range(19): # bo 19 gqtunków
        if index in indexes:
            genre_feature_vector.append(1)
        else:
            genre_feature_vector.append(0)
    return genre_feature_vector

In [26]:
genresFeatureVectorCreator(details["12"])

[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [27]:
import json

# Open the file and load the JSON data
with open('actors.json', 'r', encoding='utf-8') as file:
    actors = json.load(file)

In [33]:
ACTORS = actors

In [38]:
# ta funkcja przyjmuje movie jako json, zwraca listę aktorów z 1 jak występował dany aktor, 0 jak nie
def actorsFeatureVectorCreator(movie):
    indexes = [] # lista do przechowywania indexów, który aktor wystąpił
    for actor in getCast(movie):
        for actorFromJson in ACTORS:
            if actor == actorFromJson["name"]:
                indexes.append(actorFromJson["index"])
    actor_feature_vector = []
    for index in range(1664): # bo jest 1664 aktorów
        if index in indexes:
            actor_feature_vector.append(1)
        else:
            actor_feature_vector.append(0)
    return actor_feature_vector

In [40]:
actorsFeatureVector = actorsFeatureVectorCreator(details["12"])

In [48]:
def getDirector(movie):
    for elem in movie["credits"]["crew"]:
        if elem["job"] == "Director":
            return elem["name"]

In [49]:
getDirector(details["12"])

'Andrew Stanton'

In [45]:
# Open the file and load the JSON data
with open('directors.json', 'r', encoding='utf-8') as file:
    DIRECTORS = json.load(file)

In [50]:
# ta funkcja przyjmuje movie jako json, zwraca listę reżyserów z 1 jak występował dany aktor, 0 jak nie
def directorsFeatureVectorCreator(movie):
    indexes = [] # lista do przechowywania indexów, który reżyser wystąpił
    for directorFromJson in DIRECTORS:
        if getDirector(movie) == directorFromJson["name"]:
            indexes.append(directorFromJson["index"])
    director_feature_vector = []
    for index in range(61): # bo jest 61 reżyserów
        if index in indexes:
            director_feature_vector.append(1)
        else:
            director_feature_vector.append(0)
    return director_feature_vector

In [5]:
def getProductionCompanies(movie):
    list_of_production_companies = []
    for elem in movie["production_companies"]:
        list_of_production_companies.append(elem["name"])
    return list_of_production_companies

In [17]:
getProductionCompanies(details["13"])

['Paramount', 'The Steve Tisch Company', 'Wendy Finerman Productions']

In [9]:
# Open the file and load the JSON data
with open('json/production_companies.json', 'r', encoding='utf-8') as file:
    PRODUCTION_COMPANIES = json.load(file)

In [12]:
# ta funkcja przyjmuje movie jako json, zwraca listę production_companies z 1 jak występowała dana production_comapny, 0 jak nie
def productionCompaniesFeatureVectorCreator(movie):
    indexes = [] # lista do przechowywania indexów, która production_company wystąpiła
    for productionCompany in getProductionCompanies(movie):
        for productionCompanyFromJson in PRODUCTION_COMPANIES:
            if productionCompany == productionCompanyFromJson["name"]:
                indexes.append(productionCompanyFromJson["index"])
    production_company_feature_vector = []
    for index in range(353): # bo jest 353 production_companies
        if index in indexes:
            production_company_feature_vector.append(1)
        else:
            production_company_feature_vector.append(0)
    return production_company_feature_vector

In [18]:
temp = productionCompaniesFeatureVectorCreator(details["13"])

In [21]:
# Find all occurrences using list comprehension
indices = [index for index, value in enumerate(temp) if value == 1]
print(indices)

[122, 209, 234]


In [22]:
print(PRODUCTION_COMPANIES[122]["name"])
print(PRODUCTION_COMPANIES[209]["name"])
print(PRODUCTION_COMPANIES[234]["name"])

The Steve Tisch Company
Wendy Finerman Productions
Paramount


In [25]:
# Open the file and load the JSON data
with open('json/production_countries.json', 'r', encoding='utf-8') as file:
    PRODUCTION_COUNTRIES = json.load(file)

In [23]:
def getProductionCountries(movie):
    list_of_production_countries = []
    for elem in movie["production_countries"]:
        list_of_production_countries.append(elem["name"])
    return list_of_production_countries

In [24]:
getProductionCountries(details["62"])

['United Kingdom', 'United States of America']

In [35]:
# ta funkcja przyjmuje movie jako json, zwraca listę production_countries z 1 jak występowała dane production_country, 0 jak nie
def productionCountriesFeatureVectorCreator(movie):
    indexes = [] # lista do przechowywania indexów, która production_company wystąpiła
    for productionCountry in getProductionCountries(movie):
        for productionCountryFromJson in PRODUCTION_COUNTRIES:
            if productionCountry == productionCountryFromJson["name"]:
                indexes.append(productionCountryFromJson["index"])
    production_country_feature_vector = []
    for index in range(23): # bo jest 353 production_companies
        if index in indexes:
            production_country_feature_vector.append(1)
        else:
            production_country_feature_vector.append(0)
    return production_country_feature_vector

In [36]:
temp = productionCountriesFeatureVectorCreator(details["62"])

In [37]:
# Find all occurrences using list comprehension
indices = [index for index, value in enumerate(temp) if value == 1]
print(indices)

[0, 11]


In [39]:
print(PRODUCTION_COUNTRIES[0]["name"])
print(PRODUCTION_COUNTRIES[11]["name"])

United Kingdom
United States of America
